<a href="https://colab.research.google.com/github/uni-3/jupyter-notebooks/blob/master/surprise_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install surprise

     |████████████████████████████████| 3.3MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.0.6-cp36-cp36m-linux_x86_64.whl size=1683519 sha256=38f0e9ce9916bf7b8561736c73c67c267e21c0c240a3d65e6e0976d8eee22768
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
import surprise as sur
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import networkx as nx
plt.style.use('ggplot')
%matplotlib inline

In [0]:
url = 'http://files.grouplens.org/datasets/movielens/ml-100k/'

col_users = ['user_id','age','sex','occupation','zip_code']
df_users = pd.read_csv(url+'u.user', sep='|', names=col_users)

col_rat = ['user_id','movie_id','rating','unix_timestamp']
df_ratings = pd.read_csv(url+'u.data',
                       sep='\t', names=col_rat)

col_items = ['movie_id', 'movie_title' ,'release_date','video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci_Fi', 'Thriller', 'War', 'Western']
df_items = pd.read_csv(url+'u.item',
                   sep='|', names=col_items, encoding='latin-1')


In [0]:
# load_from_dfに必要、ratingの範囲を指定する
reader = sur.Reader(rating_scale=(1,5))

sur_data = sur.Dataset.load_from_df(df_ratings[['user_id','movie_id','rating']], reader)

In [7]:
sur_data.df.head(2)

,user_id,movie_id,rating
0,196,242,3
1,186,302,3


In [8]:
# アルゴリズムとデータの定義

sur_svd = sur.SVD(random_state = 1,
              reg_all=0.2, # using some regularization
              n_epochs=20,
              n_factors=100) # number of latent factors to retain

trainset_full = sur_data.build_full_trainset()

# 学習
sur_svd.fit(trainset_full)


In [9]:
# 各手法で評価
trainsetfull_build = trainset_full.build_testset()
predictions_full = sur_svd.test(trainsetfull_build)

# 評価用（ここでは学習データを使う）
rmse = sur.accuracy.rmse(predictions_full)
mae = sur.accuracy.mae(predictions_full)

RMSE: 0.9168
MAE:  0.7305


In [10]:
predictions_full[0:2]

[Prediction(uid=196, iid=242, r_ui=3.0, est=3.911793041493128, details={'was_impossible': False}),
 Prediction(uid=196, iid=393, r_ui=4.0, est=3.4204953983043573, details={'was_impossible': False})]

In [0]:
# 値と、評価結果を格納したデータフレーム作成

df_pred = pd.DataFrame([(x.r_ui, x.est) for x in predictions_full], 
                       columns=['Rating','Predicted'])

In [0]:
# 学習データとテストデータを分けて学習させる
import random
raw_ratings = sur_data.raw_ratings
np.random.seed(0)

random.shuffle(raw_ratings)

In [0]:
# 9:1で分ける

threshold = int(0.9 * len(raw_ratings))
train_raw_ratings = raw_ratings[:threshold]
test_raw_ratings = raw_ratings[threshold:]

sur_data.raw_ratings = train_raw_ratings

In [14]:
len(sur_data.raw_ratings)

90000

In [15]:
# cross valudationを行う
sur_svd = sur.SVD(random_state = 1)
cv_results = sur.model_selection.cross_validate(sur_svd, sur_data, measures=['RMSE','MAE'], cv=5)
pd.DataFrame(cv_results)

,test_rmse,test_mae,fit_time,test_time
0,0.949523,0.748006,5.471793,0.275309
1,0.935327,0.738424,5.410445,0.139594
2,0.937613,0.742244,5.389954,0.143568
3,0.942280,0.742586,5.402754,0.140658
4,0.942708,0.744475,5.414587,0.266189


In [0]:
# gird searchをやってみる
param_grid = {'n_epochs': [15,20,25],
            'lr_all': [0.002, 0.02, 0.2, 2]}

grid_search = sur.model_selection.GridSearchCV(sur.SVD,
                                              param_grid,
                                              measures=['rmse'],
                                              cv = 3,
                                              refit = True)

grid_search.fit(sur_data)

# bestのモデルを取り出す
sur_svd = grid_search.best_estimator['rmse']

In [17]:
# 学習用データで学習させる

trainset = sur_data.build_full_trainset()
sur_svd.fit(trainset)

# 評価
trainset_build = trainset.build_testset()
predictions_train = sur_svd.test(trainset_build)
print('Training score (rated items):')
sur.accuracy.rmse(predictions_train)

# test用データで評価
testset = sur_data.construct_testset(test_raw_ratings)
predictions_test = sur_svd.test(testset)
print('Test score (rated items)')
sur.accuracy.rmse(predictions_test)

# ratingされていないデータを使って、予測
no_ratings = trainset.build_anti_testset()
predictions_no_ratings = sur_svd.test(no_ratings)
print('Test score (unrated items):')
sur.accuracy.rmse(predictions_no_ratings)

Training score (rated items):
RMSE: 0.8367
Test score (rated items)
RMSE: 0.9428
Test score (unrated items):
RMSE: 0.5167


0.5167264953695723

In [18]:
# 全結果について、サンプルを比較
print(predictions_train[0])
print(predictions_train[1])
print("")
print(predictions_test[0])
print(predictions_test[1])
print("")
print(predictions_no_ratings[0])
print(predictions_no_ratings[1])

user: 903        item: 46         r_ui = 4.00   est = 3.66   {'was_impossible': False}
user: 903        item: 333        r_ui = 4.00   est = 3.56   {'was_impossible': False}

user: 663        item: 259        r_ui = 2.00   est = 2.83   {'was_impossible': False}
user: 405        item: 1119       r_ui = 3.00   est = 1.75   {'was_impossible': False}

user: 903        item: 205        r_ui = 3.53   est = 4.21   {'was_impossible': False}
user: 903        item: 219        r_ui = 3.53   est = 3.50   {'was_impossible': False}



KNNでやってみる。collaborative filterとは、違って、近傍のユーザのratingの和をとる。すなわち、アイテム群のratingの和が取れる

In [0]:
# アイテム同士の近さを取る
# @see https://surprise.readthedocs.io/en/stable/knn_inspired.html
sim_options = {'name': 'cosine',
               'user_based': False}

sur_knn = sur.KNNWithMeans(sim_options = sim_options)


In [21]:
sur_knn.fit(trainset)

predictions_train = sur_knn.test(trainset_build)
print('Biased accuracy on train group:')
sur.accuracy.rmse(predictions_train)

predictions_test = sur_knn.test(testset)
print('Unbiased accuracy on test group:')
sur.accuracy.rmse(predictions_test)

predictions_no_ratings = sur_knn.test(no_ratings)
print('Biased accuracy on no rated group:')
sur.accuracy.rmse(predictions_no_ratings)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Biased accuracy on train group:
RMSE: 0.7833
Unbiased accuracy on test group:
RMSE: 0.9302
Biased accuracy on no rated group:
RMSE: 0.9691


0.9690796104367222

In [0]:
# ユーザ同士の近さをだす

sim_options = {'name': 'cosine',
               'user_based': True}

In [0]:
sur_knn = sur.KNNWithMeans(sim_options = sim_options)


In [25]:
sur_knn.fit(trainset)

predictions_train = sur_knn.test(trainset_build)
print('Biased accuracy on train group:')
sur.accuracy.rmse(predictions_train)

predictions_test = sur_knn.test(testset)
print('Unbiased accuracy on test group:')
sur.accuracy.rmse(predictions_test)

predictions_no_ratings = sur_knn.test(no_ratings)
print('Biased accuracy on no rated group:')
sur.accuracy.rmse(predictions_no_ratings)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Biased accuracy on train group:
RMSE: 0.8244
Unbiased accuracy on test group:
RMSE: 0.9457
Biased accuracy on no rated group:
RMSE: 0.7690


0.7690431776897775

In [0]:
# 評価
from collections import defaultdict

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''k個分の評価を取る'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    f1s = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

        # F1@K:
        f1s[uid] = 2*precisions[uid]*recalls[uid] / \
            (precisions[uid]+recalls[uid]
             ) if (precisions[uid]+recalls[uid]) != 0 else 1

    return precisions, recalls, f1s

In [0]:
precisions, recalls, f1s = precision_recall_at_k(predictions_train, k=5, threshold=4)


In [28]:
# モデルの評価値 train
print(np.mean(list(precisions.values())))
print(np.mean(list(recalls.values())))
print(np.mean(list(f1s.values())))

0.9909155178508308
0.11513510525973703
0.1898208704011399


In [0]:
# モデルの評価値 test
precisions, recalls, f1s = precision_recall_at_k(predictions_test, k=5, threshold=4)


In [30]:

print(np.mean(list(precisions.values())))
print(np.mean(list(recalls.values())))
print(np.mean(list(f1s.values())))

0.9033732317736671
0.3249044733078165
0.3703663846094594


In [0]:
# 似ているもの取得
def get_top_n(predictions, n=10):

    # 予測結果取得    
    top_n = {}
    for uid, iid, true_r, est, _ in predictions:
        if uid in top_n.keys():
            top_n[uid].append((iid, est))
        else:
            top_n[uid] = [(iid,est)]

    # ratingごとの取得
    
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [0]:
# trainのpredictionを取得
top_n = get_top_n(predictions_train, n=10)


In [0]:
# ユーザがアイテムにつけるratingの予測取得
def get_ratings(user, top_n, ratings, items):

    filtered = ratings[user][top_n:]
    filtered.sort(key=lambda tup: tup[1], reverse=True)

    top_df = pd.DataFrame(filtered, columns=['movie_id','prediction'])
    top_df = pd.merge(left=top_df, right=items[['movie_id','movie_title']], on='movie_id')
    
    return top_df

In [34]:
get_ratings(10, 5, top_n, df_items)


,movie_id,prediction,movie_title
0,603,4.916034,Rear Window (1954)
1,100,4.915548,Fargo (1996)
2,474,4.904074,Dr. Strangelove or: How I Learned to Stop Worr...
3,134,4.884801,Citizen Kane (1941)
4,12,4.883523,"Usual Suspects, The (1995)"


In [40]:
# topnに入ったアイテムがアイテム全体の何割を占めているか
top_n_list = []
for key in top_n.keys():
    for val in top_n[key]:
        top_n_list.append([key, val[0], val[1]])
        
df_top = pd.DataFrame(top_n_list, columns=['uid', 'iid', 'est'])
df_top.head()

,uid,iid,est
0,903,357,4.694123
1,903,127,4.667889
2,903,50,4.616572
3,903,272,4.590428
4,903,318,4.578479


In [41]:
# カバレッジの割合
len(df_top.iid.unique())/len(df_ratings['movie_id'].unique())


0.2907253269916766

In [38]:
print(len(df_ratings['movie_id'].unique()))
print(len(df_ratings['user_id'].unique()))

1682
943
